In [ ]:
import torch
import matplotlib.pyplot as plt
from pytorch3d.io import load_objs_as_meshes
from pathlib import Path
from neural_poisson.data.prepare import extract_surface_data, uniform_sphere_cameras

# settings
image_size = 512
segments = 10
device = "cuda"
model_id = "1a04e3eab45ca15dd86060f189eb133"

# define the paths
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCoreTiny/02691156"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

# load the mesh and the cameras
mesh = load_objs_as_meshes([shapenet_path], device=device)
cameras = uniform_sphere_cameras(segments=segments, device=device)

# visualize a camera
elev = 8
azim = 1
data = extract_surface_data(
    camera=cameras[elev + azim * segments],
    mesh=mesh,
    image_size=image_size,
)
normal = torch.clip(data["normal_map"][0], 0.0, 1.0)
plt.imshow(normal.detach().cpu().numpy())

In [ ]:
normals = []
points = []
for camera in cameras[:20]:
    data = extract_surface_data(camera=camera, mesh=mesh, image_size=image_size)
    normals.append(data["normals"])
    points.append(data["points"])
normals = torch.cat(normals) 
points = torch.cat(points) 

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points.detach().cpu().numpy())
pcd.normals = o3d.utility.Vector3dVector(normals.detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
import random
random.choice([1, 2, 3])

In [ ]:
torch.randperm(100)[:10]

In [ ]:
from neural_poisson.data.shapenet import ShapeNetCoreDataset

dataset = ShapeNetCoreDataset(
    path=
)